<a href="https://colab.research.google.com/github/yoh6ly/Yohaly/blob/main/MODELO_(MONITOREO_DE_ENERGIA_RENOVABLE_Y_ELECTRICA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Librerías
import csv
import datetime
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
import serial   # Librería para comunicación serial con Arduino/ESP

# Configuración del puerto serial
arduino = serial.Serial('COM3', 115200)

# Lectura de sensores desde Arduino
def leer_sensores():
    linea = arduino.readline().decode().strip()
    voltaje, corriente, potencia, humedad, temperatura = map(float, linea.split(","))
    return voltaje, corriente, potencia, humedad, temperatura

# Registro de datos
def registrar_datos(nombre_archivo, v, c, p, h, t):
    fecha = datetime.datetime.now()
    with open(nombre_archivo, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([fecha, v, c, p, h, t])

# Alertas básicas
def verificar_alertas(p, h, v, equipo="Ventilador"):
    if p > 80 or v < 11 or h > 80:
        arduino.write(f"1 {equipo}\n".encode())  # Señal de alerta
        print(f"Alerta detectada en {equipo}")
    else:
        arduino.write(b"0\n")

# Gráficos automáticos
def graficar_reporte(nombre_archivo):
    datos = pd.read_csv(nombre_archivo, header=None,
                        names=["Fecha","Voltaje","Corriente","Potencia","Humedad","Temperatura"])
    plt.figure(figsize=(10,6))
    plt.plot(datos["Fecha"], datos["Potencia"], label="Potencia (W)", color="orange")
    plt.plot(datos["Fecha"], datos["Voltaje"], label="Voltaje (V)", color="blue")
    plt.plot(datos["Fecha"], datos["Corriente"], label="Corriente (A)", color="green")
    plt.xticks(rotation=45)
    plt.title("Reporte horario de energía")
    plt.xlabel("Tiempo")
    plt.ylabel("Valores")
    plt.legend()
    plt.tight_layout()
    plt.savefig(nombre_archivo.replace(".csv",".png"))
    plt.close()

# IA para anomalías
def detectar_anomalias(nombre_archivo, equipo="Ventilador"):
    datos = pd.read_csv(nombre_archivo, header=None,
                        names=["Fecha","Voltaje","Corriente","Potencia","Humedad","Temperatura"])
    modelo = IsolationForest(contamination=0.1, random_state=42)
    X = datos[["Voltaje","Corriente","Potencia","Humedad","Temperatura"]]
    etiquetas = modelo.fit_predict(X)
    datos["Estado"] = ["Anómalo" if e==-1 else "Normal" for e in etiquetas]
    datos.to_csv(nombre_archivo.replace(".csv","_anomalias.csv"), index=False)
    if etiquetas[-1] == -1:
        arduino.write(f"1 {equipo}\n".encode())
        print(f"Anomalía detectada → Falla en {equipo}")
    else:
        arduino.write(b"0\n")

# Exportar archivo diario
def exportar_diario(nombre_csv):
    datos = pd.read_csv(nombre_csv, header=None,
                        names=["Fecha","Voltaje","Corriente","Potencia","Humedad","Temperatura"])
    datos.to_excel(nombre_csv.replace(".csv",".xlsx"), index=False)
    fig, ax = plt.subplots(figsize=(12,4))
    ax.axis('tight'); ax.axis('off')
    tabla = ax.table(cellText=datos.head(20).values, colLabels=datos.columns, loc='center')
    plt.savefig(nombre_csv.replace(".csv",".pdf"))
    plt.close()

# Ciclo de monitoreo (24h con reportes cada hora)
def ciclo_monitoreo(equipo="Ventilador"):
    archivo_diario = f"diario_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
    tiempo_inicio_dia = time.time()

    while time.time() - tiempo_inicio_dia < 86400:  # 24h
        archivo_hora = f"reporte_{datetime.datetime.now().strftime('%Y%m%d_%H')}.csv"
        tiempo_inicio_hora = time.time()

        while time.time() - tiempo_inicio_hora < 3600:  # 1h
            v, c, p, h, t = leer_sensores()
            registrar_datos(archivo_hora, v, c, p, h, t)
            registrar_datos(archivo_diario, v, c, p, h, t)
            verificar_alertas(p, h, v, equipo)
            time.sleep(5)

        graficar_reporte(archivo_hora)
        detectar_anomalias(archivo_hora, equipo)

    exportar_diario(archivo_diario)

# Ejecutar ciclo del equipo
ciclo_monitoreo("Ventilador")
ciclo_monitoreo("Motor")


SerialException: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'